In [1]:
import numpy as np
#import tensorflow as tf
#from tensorflow.python.client import device_lib
import pandas as pd
from zipfile import ZipFile
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
#from skimage import exposure
#from skimage.color import rgb2gray
import platform
import sys
from datetime import datetime
%matplotlib inline

In [2]:
driving = "driving_01"

input_files = []

# read CSV file with steering angle
csvdata = pd.read_csv("./data/" + driving + ".csv", header=None)
nlabels = csvdata.shape[0]

# original shape
owidth = 320
oheight = 160

# crop
stripetop = 60
stripebot = 20

# 
# Images are in a Zip file, easier to upload to github
# 
with ZipFile("./data/" + driving + ".zip") as f:
    image_list = f.namelist()
    
    # image size is given as matrix, row X columns
    shape = (nlabels, oheight, owidth,  3)
    
    # CROP
    # cropping before training saves memory
    # but leaving the crop to the model allows better autonomous mode on the track
    
    # loop through csv lines
    #X_data = np.zeros((3*nlabels, oheight, owidth,  3), np.float)
    #y_data = np.zeros(3*nlabels, np.float)
    #for row in csvdata.iterrows():
    #    pass
    
    X_data = np.zeros(shape, np.float)
    y_data = np.zeros(nlabels, np.float)
    for k in range(1, len(image_list)):
        image = image_list[k]
        if image[11:17] == 'center':
            #ir = mpimg.imread(f.open(image), format='jpeg')
            #print(ir.shape, image)
            X_data[k-1, :, :, :] = mpimg.imread(f.open(image), format='jpeg')


y_data = y_data + csvdata.iloc[:, 3]
print("Numpy data:", X_data.shape)
print(X_data.shape[0], "images of", X_data.shape[1], "x", X_data.shape[2], "RGB")

Numpy data: (70, 160, 320, 3)
70 images of 160 x 320 RGB


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.33)
print(X_train.shape)


(46, 160, 320, 3)


continue

Could use sample data (provided) as test:

In [4]:
#testdata = pd.read_csv("./sample-data/driving.csv", header=None)
#y_test = y_test + testdata.iloc[:, 3]


# Model Architecture

The proposed model has 5 convolutional layers and 4 fully connected.

In [5]:
from keras.models import load_model, Sequential
from keras.layers import Dense, Flatten, Conv2D, Lambda, MaxPooling2D
from keras.layers import Cropping2D
from keras.callbacks import EarlyStopping
from keras import __version__ as keras_version
import h5py

Using TensorFlow backend.


In [7]:
model = Sequential()

# CROP images
model.add(Cropping2D(cropping=((stripetop, stripebot), (0,0)), input_shape=(oheight, owidth, 3)))

# normalization and mean centering
model.add(Lambda(lambda x: (x / 255.0) - 0.5))

# layer 1 - convolutional
#model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(X_data.shape[1], X_data.shape[2], X_data.shape[3])))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D())

# layer 2
#model.add(Conv2D(256, (3, 3), activation='relu'))

# layer 3
#model.add(Conv2D(128, (3, 3), activation='relu'))

# layer 4
#model.add(Conv2D(64, (3, 3), activation='relu'))

# layer 5
#model.add(Conv2D(32, (3, 3), activation='relu'))

#model.add(Dropout(0.5))
model.add(Flatten())

# layer 6
#model.add(Dense(1))
#model.add(Dropout(0.5))

# layer 7
#model.add(Dense(1000, activation='relu'))

# layer 8
#model.add(Dense(100, activation='relu'))
#model.add(Dense(10, activation='relu'))

# layer 9
model.add(Dense(1))

In [8]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cropping2d_2 (Cropping2D)    (None, 80, 320, 3)        0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 80, 320, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 78, 318, 16)       448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 39, 159, 16)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 99216)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 99217     
Total params: 99,665
Trainable params: 99,665
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
ee = 1e-6
cb_earlystop = EarlyStopping(min_delta=ee, patience=3, verbose=1)

model.compile(loss='mse', optimizer='adam')
history = model.fit(X_train, y_train, 
          batch_size=32, 
          epochs=10, 
          validation_split=0.2,
          shuffle=True,
          callbacks=[cb_earlystop], 
          verbose=1)
model.save('model.h5')


Train on 36 samples, validate on 10 samples
Epoch 1/10
36/36 [==============================] - 1s - loss: 0.3396 - val_loss: 1.3101
Epoch 2/10
36/36 [==============================] - 0s - loss: 1.6806 - val_loss: 0.4166
Epoch 3/10
36/36 [==============================] - 0s - loss: 0.8365 - val_loss: 2.0098
Epoch 4/10
36/36 [==============================] - 0s - loss: 2.2515 - val_loss: 0.0673
Epoch 5/10
36/36 [==============================] - 0s - loss: 0.2474 - val_loss: 0.4733
Epoch 6/10
36/36 [==============================] - 0s - loss: 0.5500 - val_loss: 0.1695
Epoch 7/10
36/36 [==============================] - 0s - loss: 0.2727 - val_loss: 0.4668
Epoch 8/10
36/36 [==============================] - 0s - loss: 0.6853 - val_loss: 0.3986
Epoch 00007: early stopping


In [10]:
print(history.history)

NameError: name 'history' is not defined

## Evaluate

In [ ]:
from keras.models import load_model

In [ ]:
model.load_model('model.h5')
score = model.evaluate(x_test, y_test, batch_size=32)
print(score)